In [23]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santa-workshop-tour-2019/sample_submission.csv
/kaggle/input/santa-workshop-tour-2019/family_data.csv


# Read DATA

In [24]:
import numpy as np 
import pandas as pd 
import time
import os
df = pd.read_csv('/kaggle/input/santa-workshop-tour-2019/family_data.csv', index_col = 'family_id')
submission = pd.read_csv('/kaggle/input/santa-workshop-tour-2019/sample_submission.csv', index_col = 'family_id')

In [25]:
family_size_dict = df[['n_people']].to_dict()['n_people']

cols = [f'choice_{i}' for i in range(10)]
choice_dict = df[cols].to_dict()
N_DAYS = 100
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125

days = list(range(N_DAYS, 0, -1))

In [26]:
def cost_function(prediction):
    
    penalty = 0
    daily_occupancy_cost = {k: 0 for k in days}
    
    for f, d in enumerate(prediction):
        
        n = family_size_dict[f]
        choice_0 = choice_dict['choice_0'][f]
        choice_1 = choice_dict['choice_1'][f]
        choice_2 = choice_dict['choice_2'][f]
        choice_3 = choice_dict['choice_3'][f]
        choice_4 = choice_dict['choice_4'][f]
        choice_5 = choice_dict['choice_5'][f]
        choice_6 = choice_dict['choice_6'][f]
        choice_7 = choice_dict['choice_7'][f]
        choice_8 = choice_dict['choice_8'][f]
        choice_9 = choice_dict['choice_9'][f]
        daily_occupancy_cost[d] += n
        
        if d == choice_0:
            penalty += 0
        elif d == choice_1:
            penalty += 50
        elif d == choice_2:
            penalty += 50 + 9 * n
        elif d == choice_3:
            penalty += 100 + 9 * n
        elif d == choice_4:
            penalty += 200 + 9 * n
        elif d == choice_5:
            penalty += 200 + 18 * n
        elif d == choice_6:
            penalty += 300 + 18 * n
        elif d == choice_7:
            penalty += 300 + 36 * n
        elif d == choice_8:
            penalty += 400 + 36 * n
        elif d == choice_9:
            penalty += 500 + 36 * n + 199 * n
        else:
            penalty += 500 + 36 * n + 398 * n
            
    for _, v in daily_occupancy_cost.items():
        if (v > MAX_OCCUPANCY) or (v < MIN_OCCUPANCY):
            penalty += 100000000

    accounting_cost = (daily_occupancy_cost[days[0]] - 125.0) / 400.0 * daily_occupancy_cost[days[0]]**(0.5)
    accounting_cost = max(0, accounting_cost)

    yesterday_count = daily_occupancy_cost[days[0]]

    for day in days[1:]:
        today_count = daily_occupancy_cost[day]
        diff = abs(today_count - yesterday_count)
        accounting_cost += max(0, (daily_occupancy_cost[day] - 125.0) / 400.0 * daily_occupancy_cost[day]**(0.5 + diff / 50.0))
        yesterday_count = today_count
    penalty += accounting_cost

    return penalty
            

In [27]:
def calc_family_penalty(f, d, daily_occupancy_fn):
    penalty = 0
    
    n = family_size_dict[f]
    choice_0 = choice_dict['choice_0'][f]
    choice_1 = choice_dict['choice_1'][f]
    choice_2 = choice_dict['choice_2'][f]
    choice_3 = choice_dict['choice_3'][f]
    choice_4 = choice_dict['choice_4'][f]
    choice_5 = choice_dict['choice_5'][f]
    choice_6 = choice_dict['choice_6'][f]
    choice_7 = choice_dict['choice_7'][f]
    choice_8 = choice_dict['choice_8'][f]
    choice_9 = choice_dict['choice_9'][f]
    if d == choice_0:
        penalty += 0
    elif d == choice_1:
        penalty += 50
    elif d == choice_2:
        penalty += 50 + 9 * n
    elif d == choice_3:
        penalty += 100 + 9 * n
    elif d == choice_4:
        penalty += 200 + 9 * n
    elif d == choice_5:
        penalty += 200 + 18 * n
    elif d == choice_6:
        penalty += 300 + 18 * n
    elif d == choice_7:
        penalty += 300 + 36 * n
    elif d == choice_8:
        penalty += 400 + 36 * n
    elif d == choice_9:
        penalty += 500 + 36 * n + 199 * n
    else:
        penalty += 500 + 36 * n + 398 * n
    for _, v in daily_occupancy_fn.items():
        if (v > MAX_OCCUPANCY) or (v < MIN_OCCUPANCY):
            penalty += 100000000
            
    return penalty

Define a function that calculates the accounting penalty for a specific day.

In [28]:
def accounting_day(day, daily_occupancy_2):
    if(day != 100):
        diff = abs(daily_occupancy_2[day + 1] - daily_occupancy_2[day])
        return max(0, (daily_occupancy_2[day] - 125.0) / 400.0 * daily_occupancy_2[day]**(0.5 + diff / 50.0))
    else:
        return (daily_occupancy_2[day] - 125.0) / 400.0 * daily_occupancy_2[day]**(0.5)

This is starter code again. We start by using the 'cost_function' and try to move each family to a day on their list.

In [29]:
best = submission['assigned_day'].tolist()
start_score = cost_function(best)
print(start_score)
new = best.copy()


10641498.403135022


Initialize a global variable to track the daily occupancy for each day, which we can pass to the accounting_day and calc_family_penalty functions.

In [30]:
daily_occupancy = {k: 0 for k in days}

for f, d in enumerate(new):   
        n = family_size_dict[f]
        daily_occupancy[d] += n

A method that tries to move every family to every other day, and keeps them there if the cost decreases. Note that the total cost isn't calculated each time (it would take too long). Instead, only the parts of the total cost that change are calculated, before and after the move.

In [31]:
def change_family_days():
    
    global daily_occupancy
    
    for fam_id in range(5000):
            
            # try moving them to every day
            for i in range(1, 101):
                
                fam1_day = new[fam_id]
                
                # accounting cost for the family's current day, before switching them
                accounting_current_before_1 = accounting_day(fam1_day, daily_occupancy)
                
                # accounting cost for the day before the family's current day, before switching them
                if((fam1_day != 1) & (i != fam1_day - 1)):
                    accounting_current_before_2 = accounting_day(fam1_day - 1, daily_occupancy)
                else: 
                    accounting_current_before_2 = 0
                    # accounting cost for the day we're going to try to move the family to, before switching them
                accounting_future_before_1 = accounting_day(i, daily_occupancy)
                
                # accounting cost for the day before the day we're going to try to move the family to, before switching them
                if((i != 1) & (fam1_day != i - 1)):
                    accounting_future_before_2 = accounting_day(i - 1, daily_occupancy)
                else: 
                    accounting_future_before_2 = 0
                    
                # the penalty based on the family's choices, before moving them
                fam1_penalty_before = calc_family_penalty(fam_id, fam1_day, daily_occupancy)
                # the total cost of the things that will change after we move the family
                before = accounting_current_before_1 + accounting_current_before_2 + fam1_penalty_before + accounting_future_before_1 + accounting_future_before_2

                # move the family by updating the daily occupancy
                daily_occupancy_temp = daily_occupancy.copy()
                daily_occupancy_temp[fam1_day] = daily_occupancy_temp[fam1_day] - family_size_dict[fam_id]
                daily_occupancy_temp[i] = daily_occupancy_temp[i] + family_size_dict[fam_id]

                # accounting cost for the day the family is moving to, after we've moved them
                accounting_future_after_1 = accounting_day(i, daily_occupancy_temp)
                
                # accounting cost for the day before the one the family is moving to, after we've moved them
                if((i != 1) & (fam1_day != i - 1)):
                    accounting_future_after_2 = accounting_day(i - 1, daily_occupancy_temp)
                else: 
                    accounting_future_after_2 = 0
                
                # accounting cost for the day the family started on, after we've moved them
                accounting_day_current_after_1 = accounting_day(fam1_day, daily_occupancy_temp)
                
                # accounting cost for the day before the one the family started on, after we've moved them
                if((fam1_day != 1) & (i != fam1_day - 1)):
                    accounting_current_after_2 = accounting_day(fam1_day - 1, daily_occupancy_temp)
                else: 
                    accounting_current_after_2 = 0
                
                # the penalty based on the family's choices, after moving them
                fam1_penalty_post = calc_family_penalty(fam_id, i, daily_occupancy_temp)
                
                # the total cost of the things that will change after we've moved the family
                after = accounting_future_after_1 + accounting_future_after_2 + fam1_penalty_post + accounting_day_current_after_1 + accounting_current_after_2

                # if the overall cost has decreased, move them
                if(before > after):
                    print(f'Switching {fam_id} from day {fam1_day} to day {i}')
                    new[fam_id] = i 
                    daily_occupancy = daily_occupancy_temp.copy()

A method that tries to switch each family with each other family, and keeps them there if the cost decreases. Note that the total cost isn't calculated each time (it would take too long). Instead, only the parts of the total cost that change are calculated, before and after the move.

In [32]:
def swap_families():
    
    global daily_occupancy
    
    for fam_id in range(5000):
        for fam_id2 in range(5000):
            if fam_id2 == fam_id:
                continue
  
            fam1_day = new[fam_id]
            fam2_day = new[fam_id2]
            
            accounting_fam1_before_1 = accounting_day(fam1_day, daily_occupancy)
            
            if((fam1_day != 1) & (fam1_day - 1 != fam2_day)):
                accounting_fam1_before_2 = accounting_day(fam1_day - 1, daily_occupancy)
            else: 
                accounting_fam1_before_2 = 0
                
            accounting_fam2_before_1 = accounting_day(fam2_day, daily_occupancy)
            if((fam2_day != 1) & (fam2_day - 1 != fam1_day)):
                accounting_fam2_before_2 = accounting_day(fam2_day - 1, daily_occupancy)
            else:
                accounting_fam2_before_2 = 0
                
            fam1_penalty_pre = calc_family_penalty(fam_id, fam1_day, daily_occupancy)
            fam2_penalty_pre = calc_family_penalty(fam_id2, fam2_day, daily_occupancy)
            
            # The cost of everything that will change, before the families are switched
            before = accounting_fam1_before_1 + accounting_fam1_before_2 + accounting_fam2_before_1 + accounting_fam2_before_2 + fam1_penalty_pre + fam2_penalty_pre

            # Switch the families by updating the daily occupancy
            daily_occupancy_temp = daily_occupancy.copy()
            daily_occupancy_temp[fam1_day] = daily_occupancy_temp[fam1_day] - family_size_dict[fam_id] + family_size_dict[fam_id2]
            daily_occupancy_temp[fam2_day] = daily_occupancy_temp[fam2_day] - family_size_dict[fam_id2] + family_size_dict[fam_id]

            accounting_fam1_after_1 = accounting_day(fam1_day, daily_occupancy_temp)
            
            if((fam1_day != 1) & (fam1_day - 1 != fam2_day)):
                accounting_fam1_after_2 = accounting_day(fam1_day - 1, daily_occupancy_temp)
            else:
                accounting_fam1_after_2 = 0
                
            accounting_fam2_after_1 = accounting_day(fam2_day, daily_occupancy_temp)
            
            if((fam2_day != 1) & (fam2_day - 1 != fam1_day)):
                accounting_fam2_after_2 = accounting_day(fam2_day - 1, daily_occupancy_temp)
            else:
                accounting_fam2_after_2 = 0
                
            fam1_penalty_post = calc_family_penalty(fam_id, fam2_day, daily_occupancy_temp)
            fam2_penalty_post = calc_family_penalty(fam_id2, fam1_day, daily_occupancy_temp)
            after = accounting_fam1_after_1 + accounting_fam1_after_2 + accounting_fam2_after_1 + accounting_fam2_after_2 + fam1_penalty_post + fam2_penalty_post

            # If the cost decreases, swap them
            if(before > after):
                daily_occupancy = daily_occupancy_temp.copy()
                new[fam_id] = fam2_day
                new[fam_id2] = fam1_day

Now, we basically just alternate calling these functions until we've reached a point where neither of them can do anything to make the cost smaller.

In [ ]:
import random
cost = cost_function(new)
i = 0
b = 0
print('Working on minimizing cost')
print(f'current cost: {cost}')
while(True):
    i = i + 1
    change_family_days()
    new_cost = cost_function(new)
    if(new_cost < cost):
        print(f'Round {i}: Changed family days.  New cost: {new_cost}')
        cost = new_cost
        continue
    swap_families()
    new_cost = cost_function(new)
    if(new_cost < cost):
        print(f'Round {i}: Swapped families.  New cost: {new_cost}')
        cost = new_cost
        continue
    else:
        break

Switching 0 from day 100 to day 1
Switching 0 from day 1 to day 3
Switching 0 from day 3 to day 9
Switching 0 from day 9 to day 10
Switching 0 from day 10 to day 12
Switching 0 from day 12 to day 38
Switching 0 from day 38 to day 52
Switching 1 from day 99 to day 1
Switching 1 from day 1 to day 3
Switching 1 from day 3 to day 4
Switching 1 from day 4 to day 26
Switching 2 from day 98 to day 1
Switching 2 from day 1 to day 3
Switching 2 from day 3 to day 9
Switching 2 from day 9 to day 10
Switching 2 from day 10 to day 12
Switching 2 from day 12 to day 25
Switching 2 from day 25 to day 54
Switching 2 from day 54 to day 100
Switching 3 from day 97 to day 1
Switching 3 from day 1 to day 2
Switching 4 from day 96 to day 1
Switching 4 from day 1 to day 53
Switching 5 from day 95 to day 2
Switching 5 from day 2 to day 3
Switching 5 from day 3 to day 12
Switching 5 from day 12 to day 59
Switching 6 from day 94 to day 1
Switching 6 from day 1 to day 4
Switching 6 from day 4 to day 88
Switching

Switching 146 from day 42 to day 64
Switching 147 from day 53 to day 1
Switching 147 from day 1 to day 7
Switching 147 from day 7 to day 24
Switching 147 from day 24 to day 28
Switching 148 from day 52 to day 3
Switching 148 from day 3 to day 4
Switching 148 from day 4 to day 25
Switching 148 from day 25 to day 63
Switching 149 from day 51 to day 1
Switching 149 from day 1 to day 18
Switching 149 from day 18 to day 38
Switching 149 from day 38 to day 40
Switching 150 from day 50 to day 3
Switching 150 from day 3 to day 4
Switching 150 from day 4 to day 6
Switching 150 from day 6 to day 12
Switching 150 from day 12 to day 19
Switching 150 from day 19 to day 60
Switching 151 from day 49 to day 2
Switching 151 from day 2 to day 3
Switching 152 from day 48 to day 1
Switching 152 from day 1 to day 40
Switching 152 from day 40 to day 88
Switching 153 from day 47 to day 1
Switching 153 from day 1 to day 2
Switching 153 from day 2 to day 3
Switching 154 from day 46 to day 1
Switching 154 from 

Switching 516 from day 13 to day 17
Switching 516 from day 17 to day 42
Switching 517 from day 83 to day 1
Switching 517 from day 1 to day 47
Switching 517 from day 47 to day 60
Switching 518 from day 82 to day 1
Switching 518 from day 1 to day 16
Switching 518 from day 16 to day 18
Switching 518 from day 18 to day 19
Switching 518 from day 19 to day 24
Switching 519 from day 81 to day 2
Switching 519 from day 2 to day 9
Switching 520 from day 80 to day 2
Switching 520 from day 2 to day 4
Switching 521 from day 79 to day 1
Switching 521 from day 1 to day 61
Switching 521 from day 61 to day 66
Switching 522 from day 78 to day 1
Switching 522 from day 1 to day 3
Switching 522 from day 3 to day 32
Switching 523 from day 77 to day 1
Switching 523 from day 1 to day 4
Switching 524 from day 76 to day 1
Switching 524 from day 1 to day 46
Switching 524 from day 46 to day 74
Switching 525 from day 75 to day 2
Switching 525 from day 2 to day 13
Switching 525 from day 13 to day 15
Switching 525 f

Switching 591 from day 49 to day 55
Switching 592 from day 8 to day 2
Switching 592 from day 2 to day 5
Switching 592 from day 5 to day 14
Switching 592 from day 14 to day 40
Switching 593 from day 7 to day 1
Switching 593 from day 1 to day 4
Switching 593 from day 4 to day 23
Switching 594 from day 6 to day 1
Switching 594 from day 1 to day 61
Switching 594 from day 61 to day 95
Switching 595 from day 5 to day 10
Switching 595 from day 10 to day 19
Switching 596 from day 4 to day 1
Switching 596 from day 1 to day 29
Switching 596 from day 29 to day 33
Switching 596 from day 33 to day 45
Switching 596 from day 45 to day 82
Switching 596 from day 82 to day 88
Switching 597 from day 3 to day 1
Switching 597 from day 1 to day 9
Switching 597 from day 9 to day 81
Switching 598 from day 2 to day 3
Switching 598 from day 3 to day 47
Switching 599 from day 1 to day 3
Switching 599 from day 3 to day 11
Switching 599 from day 11 to day 43
Switching 600 from day 100 to day 1
Switching 600 from d

Switching 683 from day 36 to day 40
Switching 684 from day 16 to day 2
Switching 684 from day 2 to day 3
Switching 684 from day 3 to day 26
Switching 685 from day 15 to day 2
Switching 685 from day 2 to day 5
Switching 685 from day 5 to day 17
Switching 685 from day 17 to day 27
Switching 685 from day 27 to day 33
Switching 685 from day 33 to day 34
Switching 686 from day 14 to day 2
Switching 686 from day 2 to day 4
Switching 686 from day 4 to day 18
Switching 686 from day 18 to day 53
Switching 687 from day 13 to day 1
Switching 687 from day 1 to day 51
Switching 688 from day 12 to day 74
Switching 688 from day 74 to day 88
Switching 689 from day 11 to day 1
Switching 689 from day 1 to day 37
Switching 689 from day 37 to day 67
Switching 690 from day 10 to day 6
Switching 690 from day 6 to day 38
Switching 691 from day 9 to day 2
Switching 691 from day 2 to day 6
Switching 691 from day 6 to day 8
Switching 692 from day 8 to day 1
Switching 692 from day 1 to day 5
Switching 692 from d

...

Switching 3440 from day 3 to day 53
Switching 3440 from day 53 to day 88
Switching 3441 from day 16 to day 39
Switching 3454 from day 66 to day 12
Switching 3472 from day 73 to day 10
Switching 3472 from day 10 to day 24
Switching 3476 from day 2 to day 40
Switching 3476 from day 40 to day 94
Switching 3480 from day 83 to day 32
Switching 3499 from day 28 to day 10
Switching 3510 from day 24 to day 15
Switching 3514 from day 1 to day 59
Switching 3527 from day 17 to day 55
Switching 3530 from day 5 to day 49
Switching 3530 from day 49 to day 89
Switching 3540 from day 90 to day 23
Switching 3557 from day 5 to day 60
Switching 3578 from day 56 to day 82
Switching 3586 from day 82 to day 1
Switching 3590 from day 61 to day 82
Switching 3612 from day 74 to day 46
Switching 3612 from day 46 to day 47
Switching 3622 from day 38 to day 24
Switching 3623 from day 8 to day 17
Switching 3658 from day 22 to day 45
Switching 3689 from day 20 to day 45
Switching 3702 from day 4 to day 46
Switching

Switching 3020 from day 1 to day 89
Switching 3027 from day 46 to day 75
Switching 3057 from day 40 to day 1
Switching 3057 from day 1 to day 4
Switching 3059 from day 69 to day 41
Switching 3081 from day 14 to day 1
Switching 3215 from day 83 to day 25
Switching 3223 from day 12 to day 25
Switching 3233 from day 18 to day 20
Switching 3234 from day 43 to day 73
Switching 3236 from day 53 to day 30
Switching 3266 from day 80 to day 61
Switching 3280 from day 37 to day 88
Switching 3286 from day 45 to day 74
Switching 3289 from day 29 to day 12
Switching 3303 from day 52 to day 55
Switching 3332 from day 54 to day 17
Switching 3359 from day 68 to day 62
Switching 3392 from day 41 to day 96
Switching 3393 from day 47 to day 26
Switching 3424 from day 53 to day 56
Switching 3468 from day 42 to day 35
Switching 3468 from day 35 to day 39
Switching 3468 from day 39 to day 40
Switching 3473 from day 47 to day 2
Switching 3476 from day 94 to day 2
Switching 3494 from day 11 to day 97
Switchin

Switching 3810 from day 85 to day 45
Switching 3853 from day 50 to day 38
Switching 3905 from day 58 to day 61
Switching 4008 from day 83 to day 64
Switching 4204 from day 46 to day 24
Switching 4210 from day 67 to day 10
Switching 4273 from day 22 to day 5
Switching 4405 from day 40 to day 62
Switching 4425 from day 3 to day 79
Switching 4431 from day 67 to day 30
Switching 4506 from day 68 to day 48
Switching 4527 from day 82 to day 79
Switching 4565 from day 31 to day 3
Switching 4566 from day 42 to day 26
Switching 4584 from day 38 to day 90
Switching 4621 from day 79 to day 2
Switching 4621 from day 2 to day 6
Switching 4621 from day 6 to day 19
Switching 4621 from day 19 to day 82
Switching 4624 from day 79 to day 88
Switching 4686 from day 24 to day 80
Switching 4690 from day 30 to day 38
Switching 4816 from day 67 to day 24
Switching 4821 from day 18 to day 33
Switching 4823 from day 77 to day 2
Switching 4823 from day 2 to day 6
Switching 4823 from day 6 to day 24
Switching 48

Switching 3170 from day 78 to day 32
Switching 3289 from day 12 to day 29
Switching 3302 from day 22 to day 54
Switching 3404 from day 13 to day 31
Switching 3415 from day 51 to day 80
Switching 3447 from day 52 to day 38
Switching 3456 from day 54 to day 34
Switching 3468 from day 40 to day 39
Switching 3476 from day 2 to day 40
Switching 3572 from day 60 to day 42
Switching 3587 from day 5 to day 48
Switching 3605 from day 39 to day 55
Switching 3607 from day 3 to day 33
Switching 3742 from day 58 to day 27
Switching 3773 from day 51 to day 17
Switching 3877 from day 48 to day 40
Switching 3973 from day 29 to day 2
Switching 4106 from day 47 to day 12
Switching 4106 from day 12 to day 67
Switching 4147 from day 33 to day 17
Switching 4210 from day 10 to day 16
Switching 4399 from day 67 to day 8
Switching 4404 from day 8 to day 10
Switching 4444 from day 15 to day 45
Switching 4455 from day 7 to day 5
Switching 4613 from day 27 to day 12
Switching 4640 from day 5 to day 75
Switching 

Switching 4193 from day 81 to day 7
Switching 4269 from day 33 to day 76
Switching 4413 from day 82 to day 10
Switching 4413 from day 10 to day 51
Switching 4452 from day 10 to day 50
Switching 4817 from day 55 to day 38
Switching 4824 from day 76 to day 7
Switching 4824 from day 7 to day 17
Switching 4824 from day 17 to day 31
Switching 4824 from day 31 to day 66
Switching 4844 from day 25 to day 45
Switching 4857 from day 8 to day 94
Switching 4898 from day 41 to day 46
Switching 4908 from day 49 to day 18
Switching 4939 from day 51 to day 25
Switching 16 from day 50 to day 46
Switching 74 from day 82 to day 28
Switching 82 from day 32 to day 8
Switching 244 from day 35 to day 39
Switching 316 from day 53 to day 44
Switching 548 from day 88 to day 22
Switching 607 from day 81 to day 51
Switching 654 from day 88 to day 10
Switching 819 from day 38 to day 32
Switching 843 from day 14 to day 88
Switching 868 from day 52 to day 33
Switching 1250 from day 13 to day 38
Switching 1363 from 

In [ ]:
print('making submission')
submission['assigned_day'] = new

In [ ]:
submission.to_csv('submission.csv')